<a href="https://colab.research.google.com/github/RishavJ7/CSS-projects/blob/main/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# === 🗂️ Mount Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === 📦 Unzip Dataset ===
!unzip -q "/content/drive/MyDrive/Copy of dry_oily_normal.zip" -d "/content/dataset"

# === 📁 Set Parameters ===
dataset_path = "/content/dataset/Oily-Dry-Skin-Types"  # Adjust folder name if needed
img_size = (128, 128)
batch_size = 32

# === 🧪 Data Augmentation and Preprocessing ===
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.3,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

# === 📥 Load Train/Val Data ===
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# === 🧠 CNN Model ===
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_size[0], img_size[1], 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

# === ⚙️ Compile Model ===
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# === ⛔ Callbacks to Prevent Overfitting ===
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# === 🚀 Train Model ===
history = model.fit(
    train_data,
    epochs=30,
    validation_data=val_data,
    callbacks=[early_stop, reduce_lr]
)

# === 📊 Evaluate Model ===
val_loss, val_accuracy = model.evaluate(val_data)
print("✅ Validation Accuracy:", val_accuracy)
print("❌ Validation Loss:", val_loss)

# === 🔍 Scan and Predict Image ===
img_path = "/content/dataset/Oily-Dry-Skin-Types/test/oily/100.jpg"  # Change if needed

img = cv2.imread(img_path)
img = cv2.resize(img, img_size)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

prediction = model.predict(img)
predicted_class = np.argmax(prediction)
class_labels = list(train_data.class_indices.keys())

print("🔮 Predicted Skin Type:", class_labels[predicted_class])


Mounted at /content/drive
Found 2523 images belonging to 3 classes.
Found 629 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.4387 - loss: 1.4648 - val_accuracy: 0.8760 - val_loss: 3.8015 - learning_rate: 0.0010
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.6569 - loss: 0.8740 - val_accuracy: 0.8760 - val_loss: 1.7451 - learning_rate: 0.0010
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy: 0.7854 - loss: 0.7181 - val_accuracy: 0.8760 - val_loss: 1.2178 - learning_rate: 0.0010
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 195s 2s/step - accuracy: 0.8178 - loss: 0.6262 - val_accuracy: 0.8760 - val_loss: 0.8653 - learning_rate: 0.0010
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.8347 - loss: 0.5925 - val_accuracy: 0.8760 - val_loss: 0.7161 - learning_rate: 0.0010
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.8438 - loss: 0.5598 - val_accuracy: 0.8760 - val_loss: 0.5100 - learning_rate: 0.0010
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.8561 - loss: 0.5142 - val_acc

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
